# [Backpropagation](https://pennylane.ai/qml/demos/tutorial_backprop.html)

The parameter-shift rule states that, given a variational quantum
circuit $U(\boldsymbol
\theta)$ composed of parametrized Pauli rotations, and some measured
observable $\hat{B}$, the derivative of the expectation value

$$\langle \hat{B} \rangle (\boldsymbol\theta) =
\langle 0 \vert U(\boldsymbol\theta)^\dagger \hat{B} U(\boldsymbol\theta) \vert 0\rangle$$

with respect to the input circuit parameters $\boldsymbol{\theta}$ is
given by

$$\nabla_{\theta_i}\langle \hat{B} \rangle(\boldsymbol\theta)
   =  \frac{1}{2}
         \left[
             \langle \hat{B} \rangle\left(\boldsymbol\theta + \frac{\pi}{2}\hat{\mathbf{e}}_i\right)
           - \langle \hat{B} \rangle\left(\boldsymbol\theta - \frac{\pi}{2}\hat{\mathbf{e}}_i\right)
         \right].$$

Thus, the gradient of the expectation value can be calculated by
evaluating the same variational quantum circuit, but with shifted
parameter values (hence the name, parameter-shift rule!).

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from matplotlib import pyplot as plt

# set the random seed
np.random.seed(42069)

# create a device to execute the circuit on
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev, diff_method="parameter-shift")
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RZ(params[2], wires=2)

    qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring")

    qml.RX(params[3], wires=0)
    qml.RY(params[4], wires=1)
    qml.RZ(params[5], wires=2)

    qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring")
    return qml.expval(qml.PauliY(0) @ qml.PauliZ(2))

In [ ]:
# initial parameters
params = np.random.random([6], requires_grad=True)

print("Parameters:", params)
print("Expectation value:", circuit(params))

In [ ]:
fig, ax = qml.draw_mpl(circuit, decimals=2)(params)
plt.show()

In [ ]:
def parameter_shift_term(qnode, params, i):
    shifted = params.copy()
    shifted[i] += np.pi/2
    forward = qnode(shifted)  # forward evaluation

    shifted[i] -= np.pi
    backward = qnode(shifted) # backward evaluation

    return 0.5 * (forward - backward)

# gradient with respect to the first parameter
print(parameter_shift_term(circuit, params, 0))

In [ ]:
def parameter_shift(qnode, params):
    gradients = np.zeros([len(params)])

    for i in range(len(params)):
        gradients[i] = parameter_shift_term(qnode, params, i)

    return gradients

print(parameter_shift(circuit, params))

In [ ]:
grad_function = qml.grad(circuit)
print(grad_function(params)[0])